<a href="https://colab.research.google.com/github/ohhappylife/ETLCFA/blob/master/News_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# News APIs Analysis
### Purpose : 
To find a APIs that can satisfy the project requirements.

### Sources :
Bing News API, Google News API, NY Times API, News API, News Catcher API

### Result : 
<B> News API seems to be a great choice, but it has a price concerns as a service providers only allows users to use their license for free in testing/IMPL environment (https://newsapi.org/pricing)</B>

### Rationale : 
-- Bing News API: Bing News API collects small number of articles about different topics from domestic famous sources.

-- Google News (third party): Google News API collect small number of articles about different topics from CNN / Fox News. 

-- NY Times: NY Times collects small number of articles about political / international topics from NY Time.

-- News API: News API collects large number of articles about different topics from both domestic / international, famous / unfamous sources.

-- News Catcher API: News Catcher API collects moderate number of articles about different topics from international / unfamous sources.




In [1]:
from google.colab import files
uploaded = files.upload()

Saving merged_20230217.csv to merged_20230217.csv


In [2]:
import pandas as pd
df = pd.read_csv('merged_20230217.csv')
df.drop_duplicates(['camefrom', 'Title'], inplace = True)
df = df[df['camefrom'] != 'politifact']

In [3]:
a = df.groupby('camefrom')['Author'].count()

In [4]:
g1 = pd.DataFrame(a)
g1.reset_index(inplace = True)
g1['percentage'] = ((g1['Author'] / a.reset_index()['Author'].sum()) * 100).astype(str).str[:5] + "%"
g1.rename(columns={"Author": "Numbers of Articles", "camefrom": "Source"}, inplace = True)

#1. Numbers of News Articles Collected
- Keywords Used: Biden, Bruce Willis, Damar Hamlin Jacket, Frontotemporal Demantia, Google, Russia, Ryan Seacrest, Tiger Woods, Turkry, Layoffs
- Max Numbers of Articles to be collected: 1000/keywords
- Sources: Bing News, Google News, NY Times, News, News catcher
- Result: 103, 83, 138, 711, 336

In [11]:
import plotly.express as px
fig = px.bar(g1, 
             x='Source', 
             y='Numbers of Articles', 
             title = 'Number of Data Collected from Sources (N = 1,371)',
             text = 'percentage'
             )
fig.update_layout(
             xaxis_title="Sources",
             yaxis_title="Number of Articles"
             )
fig.show()

In [12]:
import numpy as np
df2 = df.groupby(['camefrom','keyword'])['Author'].count()
df2 = pd.DataFrame(df2)
df2.reset_index(inplace = True)
df2['sum'] = df2.groupby(['camefrom']).Author.transform(np.sum)
df2['percentage'] = (df2['Author'] / df2['sum']) * 100
df2.rename(columns={"camefrom": "Source"}, inplace = True)

# News Articles Collected by Keywords
All sources, excpet NY Times collect similiar numbers of articles
NY Times has more news articles about political issues (Russia war and Biden), but they do not have a lot of articles about other issues. Especically, NY Times does not cover sports / celebrity topics.

In [16]:
import plotly.express as px

fig = px.bar(df2, 
             x="Source", 
             y="percentage", 
             color="keyword", 
             title="% of Articles Collected, Based on Keywords", 
             text = 'keyword')
fig.show()

# Sources from each APIs
- Bing News API collects articles from diverse / popular sources.
- Google News API collects articles from two / popular sources (CNN / Fox News)
- NY Times API collects articles from NY Times.
- NEWS Api collects articels from extremely diverse sources.
- NEWS catcher collects articles from diverse, but non-popular (international) sources.

In [17]:
df3 = (df.groupby(['camefrom', 'Author'])['Title'].count())
df3 = pd.DataFrame(df3).reset_index()
df3 = df3[(df3['Title'] > 3)]
df3['sum'] = df3.groupby(['camefrom']).Title.transform(np.sum)
df3['percentage'] = (df3['Title'] / df3['sum']) * 100

fig = px.bar(df3, x="camefrom", 
             y="percentage", 
             color="Author", 
             title="Articles' Sources, from APIs", 
             text = 'Author')
fig.show()

# Find Duplicates
- Articles from Bing news API is normally overlapped with other sources. especially with News API and Google News.
- While News API collects more than 700 articles and have less than 50 duplicates, and Bing News API collects about 100 / Google News API collects 80 Articles, but they have about 60 duplicates / 40 duplicates.

In [34]:
df4 = df[df.duplicated(subset = ['Title'], keep = False)].sort_values(['Title']).reset_index()
df4[['Title','camefrom']].sort_values('camefrom').groupby(['Title'])['camefrom'].transform(lambda x: ','.join(x)).reset_index().groupby('camefrom').count().sort_values('index', ascending = False)
df4.rename(columns={"camefrom": "Source", "index" : "Count of Duplicated (Combination)"}, inplace = True)

,index
camefrom,
"BingNews,newsAPI",22
"BingNews,GoogleNews",20
"BingNews,GoogleNews,newsAPI",9
"BingNews,newsCatcher",8
"GoogleNews,newsCatcher",8
"BingNews,GoogleNews,newsCatcher",6
"newsAPI,newsCatcher",6
"GoogleNews,newsAPI",4
"BingNews,newsAPI,newsCatcher",3
